# Data Augmentation

The goal of this notebook is to use data augmentation to improve upon the results of the original TorNet baseline CNN model.

In this notebook, we implement the image flipping used to only add flipped versions of tornado images to the TorNet dataset.
We then take that data and re-train the original architecture of the TorNet model (albeit with our re-tuned hyperparameters
for batch size and learning rate) in order to see whether the data augmentation strategy will improve the quality of the 
model that is trained by addressing the class imbalance due to the rarity of tornadoes in the dataset.

Of note - as mentioned in the project milestone, we can't use region cropping or translation here, because the 
original data is radar data which is being measured on a slant (not top-down) and hence has a natural stretching/distortion
that occurs depending on how far away a storm cell is from the center of the radar image. If we move the data around, 
we risk introducing our own squishing artifacts on the shapes of the cells which would poison our dataset. 

In comparison, mirroring here could be very helpful (without translating the data) to generate more examples of 
tornadoes without having to find more tornado radar images. Hence, we implement that strategy here.

In [12]:
import sys

import os
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

# just the location for the input data
TORNET_DATA_INPUT_FOLDER = "/mnt/c/users/handypark/Documents/Grad_School_Courses/CS_230/tornet"

In [13]:
# Just making sure that we are indeed using GPU-based Tensorflow and not CPU-based Tensorflow.
tf.test.is_built_with_cuda()

# We tried experimental memory growth in some cases, but it didn't work out well (also crashed a lot).
"""
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
"""

'\ngpus = tf.config.list_physical_devices(\'GPU\')\nif gpus:\n  try:\n    # Currently, memory growth needs to be the same across GPUs\n    for gpu in gpus:\n      tf.config.experimental.set_memory_growth(gpu, True)\n    logical_gpus = tf.config.list_logical_devices(\'GPU\')\n    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")\n  except RuntimeError as e:\n    # Memory growth must be set before GPUs have been initialized\n    print(e)\n'

## Creating a dataset with additional mirrored tornado examples

This time around, now that we have the learning rate and batch size all tuned, we can start trying to improve the performance of the model.
Our first idea is to try using data augmentation. Since examples of tornados are rare (and comprise only a tiny portion of the data),
the hope here is to try to artificially increase the number of examples of tornadoes to help correct this class imbalance.

The initial paper used weights on different kinds of examples, but the hope here is that augmenting the data can be even more effective.
What we've done below is to take TorNet's dataset loading + creation code and introduce a flipping step, where we do the following:
- Check if the given training set image is a tornadic example.
- If not, proceed as normal.
- If so, add another copy of the image that we have flipped to the training set.

With this data augmentation, we're hoping to see some improvement in the performance of the model on the testing data in terms of AUC, just from the additional "examples" of tornados that we get a result.

In [14]:
"""
TorNet's data loading code, but now with some adjustments for data augmentation purposes.
We create `create_tf_with_flips_dataset` in order to make a dataset that has both
tornadic examples and mirrored tornadic examples.
"""
from typing import List, Dict

from tornet.data.loader import query_catalog, read_file
from tornet.data.constants import ALL_VARIABLES
from tornet.data import preprocess as pp
import numpy as np

def create_tf_with_flips_dataset(files:str,
                                 variables: List[str]=ALL_VARIABLES,
                                 n_frames:int=1,
                                 tilt_last: bool=True) -> tf.data.Dataset:
    """
    This is Tornet's main function for loading the data from the folder where it's all stored, but we've 
    made adjustments to actually add a system for flipping tornadic images.

    In this case, we add non-tornadic examples as normal, but when we encounter an example of
    a tornado, we actually add both the original tornadic example and a mirrored version
    of the tornadic example to the dataset.
    """
    assert len(files)>0
    
    # grab one file to gets keys, shapes, etc
    data = read_file(files[0],variables=variables,n_frames=n_frames, tilt_last=tilt_last)
    
    output_signature = { k:tf.TensorSpec(shape=data[k].shape,dtype=data[k].dtype,name=k) for k in data }

    # Iterate through all the files and keep track of which are tornadic examples, and add 
    # a "to-be-flipped" copy for each case of a tornado.
    file_list = []
    for f in files:
        file_list.append((f, 0))
        file_dict = read_file(f,variables=variables,n_frames=n_frames, tilt_last=tilt_last)
        if file_dict["label"] == 1:
            file_list.append((f, 1))

    # When generating the data, check to see whether we should flip the data,
    # then flip if necessary.
    def gen():
        for (f, flipper) in file_list:
            file_dict = read_file(f,variables=variables,n_frames=n_frames, tilt_last=tilt_last)
            if flipper == 1:
                # Each of the 6 different radar images gets flipped using np.flip
                for var in ['DBZ','VEL','KDP','RHOHV','ZDR','WIDTH']:
                    file_dict[var] = tf.convert_to_tensor(np.flip(file_dict[var], 1))
            yield file_dict
    ds = tf.data.Dataset.from_generator(gen,
                                        output_signature=output_signature)
    return ds
    

def preproc(ds: tf.data.Dataset,
            weights:Dict=None,
            include_az:bool=False,
            select_keys:list=None,
            tilt_last:bool=True):
    """
    This is Tornet's preprocessing function, unchanged,
    for taking the raw dataset loaded from the files (in create_tf_dataset)
    and then doing a few things:

    - Remove the time dimension (since we only care about detection at a given time t)
    - Add coordinates (so that we can run CoordConv layers later)
    - Split the data into its inputs and label outputs
    - Adding weights (if we decide to weight the data at all)

    Once the preprocessing is done, the data is basically ready to be trained on.
    """
    
    # Remove time dimension
    ds = ds.map(pp.remove_time_dim)

    # Add coordinate tensors
    ds = ds.map(lambda d: pp.add_coordinates(d,include_az=include_az,tilt_last=tilt_last,backend=tf))

    # split into X,y
    ds = ds.map(pp.split_x_y)

    # Add sample weights
    if weights:
        ds = ds.map(lambda x,y:  pp.compute_sample_weight(x,y,**weights, backend=tf) )
    
        # select keys for input
        if select_keys is not None:
            ds = ds.map(lambda x,y,w: (pp.select_keys(x,keys=select_keys),y,w))
    else:
        if select_keys is not None:
            ds = ds.map(lambda x,y: (pp.select_keys(x,keys=select_keys),y))

    return ds

In [15]:
def make_tf_with_flips_loader(data_root: str, 
                              data_type:str='train', # or 'test'
                              years: list=list(range(2013,2023)),
                              batch_size: int=128, 
                              weights: Dict=None,
                              include_az: bool=False,
                              random_state:int=1234,
                              select_keys: list=None,
                              tilt_last: bool=True,
                              from_tfds: bool=False,
                              tfds_data_version: str='1.1.0',
                              num_epochs: int=3):
    """
    We've trimmed down the make_tf_loader function from the TorNet helper
    functions to make clear which part of the loader we're using.

    Now that we have a function for making a dataset with flipped tornado
    examples added in, we make use of it here.
    """
    # get files from the catalog
    file_list = query_catalog(data_root, data_type, years, random_state)

    # make the dataset using the flipped data generator
    ds = create_tf_with_flips_dataset(file_list,variables=ALL_VARIABLES,n_frames=1, tilt_last=tilt_last) 

    # do the usual preprocessing to prepare the input for Tensorflow training
    ds = preproc(ds,weights,include_az,select_keys,tilt_last)
    ds = ds.prefetch(tf.data.AUTOTUNE)

    # this has been adjusted to include drop_remainder=True
    ds = ds.batch(batch_size, drop_remainder=True)
    return ds

## TorNet Baseline CNN Model Definition:

As stated in the comment for the code block below, this is just the TorNet model code that was used in the original paper.
We'll be using the same model for now during our data augmentation experiments.

In [16]:
"""
This is just the TorNet model code that was used in the paper.
Goal is to run this model which consists of:
- Normalizing the inputs
- Adding the coordinate information for CoordConv to work properly
- Running 4 VGG blocks which each have CoordConv2D and two MAXPOOL layers
- One last block of Conv2D layers and MAXPOOL to get the output probability
"""

from typing import Dict, List, Tuple
import numpy as np
import keras
from tornet.models.keras.layers import CoordConv2D, FillNaNs
from tornet.data.constants import CHANNEL_MIN_MAX, ALL_VARIABLES


def build_model(shape:Tuple[int]=(120,240,2),
                c_shape:Tuple[int]=(120,240,2),
                input_variables:List[str]=ALL_VARIABLES,
                start_filters:int=64,
                l2_reg:float=0.001,
                background_flag:float=-3.0,
                include_range_folded:bool=True,
                head='maxpool'):
    # Create input layers for each input_variables
    inputs = {}
    for v in input_variables:
        inputs[v]=keras.Input(shape,name=v)
    n_sweeps=shape[2]
    
    # Normalize inputs and concate along channel dim
    normalized_inputs=keras.layers.Concatenate(axis=-1,name='Concatenate1')(
        [normalize(inputs[v],v) for v in input_variables]
        )

    # Replace nan pixel with background flag
    normalized_inputs = FillNaNs(background_flag)(normalized_inputs)

    # Add channel for range folded gates 
    if include_range_folded:
        range_folded = keras.Input(shape[:2]+(n_sweeps,),name='range_folded_mask')
        inputs['range_folded_mask']=range_folded
        normalized_inputs = keras.layers.Concatenate(axis=-1,name='Concatenate2')(
               [normalized_inputs,range_folded])
        
    # Input coordinate information
    cin=keras.Input(c_shape,name='coordinates')
    inputs['coordinates']=cin

    x,c = normalized_inputs,cin
    
    x,c = vgg_block(x,c, filters=start_filters,   ksize=3, l2_reg=l2_reg, n_convs=2, drop_rate=0.1)   # (60,120)
    x,c = vgg_block(x,c, filters=2*start_filters, ksize=3, l2_reg=l2_reg, n_convs=2, drop_rate=0.1)  # (30,60)
    x,c = vgg_block(x,c, filters=4*start_filters, ksize=3, l2_reg=l2_reg, n_convs=3, drop_rate=0.1)  # (15,30)
    x,c = vgg_block(x,c, filters=8*start_filters, ksize=3, l2_reg=l2_reg, n_convs=3, drop_rate=0.1)  # (7,15)
    #x,c = vgg_block(x,c, filters=8*start_filters, ksize=3, l2_reg=l2_reg, n_convs=3)  # (3,7)
    
    if head=='mlp':
        # MLP head
        x = keras.layers.Flatten()(x) 
        x = keras.layers.Dense(units = 4096, activation ='relu')(x) 
        x = keras.layers.Dense(units = 2024, activation ='relu')(x) 
        output = keras.layers.Dense(1)(x)
    elif head=='maxpool':
        # Per gridcell
        x = keras.layers.Conv2D(filters=512, kernel_size=1,
                          kernel_regularizer=keras.regularizers.l2(l2_reg),
                          activation='relu')(x)
        x = keras.layers.Conv2D(filters=256, kernel_size=1,
                          kernel_regularizer=keras.regularizers.l2(l2_reg),
                          activation='relu')(x)
        x = keras.layers.Conv2D(filters=1, kernel_size=1,name='heatmap')(x)
        # Max in scene
        output = keras.layers.GlobalMaxPooling2D()(x)

    return keras.Model(inputs=inputs,outputs=output)


def vgg_block(x,c, filters=64, ksize=3, n_convs=2, l2_reg=1e-6, drop_rate=0.0):

    for _ in range(n_convs):
        x,c = CoordConv2D(filters=filters,
                          kernel_size=ksize,
                          kernel_regularizer=keras.regularizers.l2(l2_reg),
                          padding='same',
                          activation='relu')([x,c])
    x = keras.layers.MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
    c = keras.layers.MaxPool2D(pool_size =2, strides =2, padding ='same')(c)
    if drop_rate>0:
        x = keras.layers.Dropout(rate=drop_rate)(x)
    return x,c


def normalize(x,
              name:str):
    """
    Channel-wise normalization using known CHANNEL_MIN_MAX
    """
    min_max = np.array(CHANNEL_MIN_MAX[name]) # [2,]
    n_sweeps=x.shape[-1]
    
    # choose mean,var to get approximate [-1,1] scaling
    var=((min_max[1]-min_max[0])/2)**2 # scalar
    var=np.array(n_sweeps*[var,])    # [n_sweeps,]
    
    offset=(min_max[0]+min_max[1])/2    # scalar
    offset=np.array(n_sweeps*[offset,]) # [n_sweeps,]

    return keras.layers.Normalization(mean=offset,
                                      variance=var,
                                      name='Normalize_%s' % name)(x)

## Running the Model

Ok, we've got all of the TorNet model components figured out and imported (using the helper functions, etc.).
[Even getting that working turned out to be tricky - there were dependency issues to resolve, and while we'd 
like to just be able to import the functions rather than re-copying them here again, that wasn't working so well.]

In [17]:
model = build_model()

In [18]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ DBZ (InputLayer)    │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ VEL (InputLayer)    │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ KDP (InputLayer)    │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ RHOHV (InputLayer)  │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ZDR (InputLayer)    │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ WIDTH (InputLayer)  │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalize_DBZ       │ (None, 120, 240,  │          0 │ DBZ[0][0]         │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalize_VEL       │ (None, 120, 240,  │          0 │ VEL[0][0]         │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalize_KDP       │ (None, 120, 240,  │          0 │ KDP[0][0]         │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalize_RHOHV     │ (None, 120, 240,  │          0 │ RHOHV[0][0]       │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalize_ZDR       │ (None, 120, 240,  │          0 │ ZDR[0][0]         │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalize_WIDTH     │ (None, 120, 240,  │          0 │ WIDTH[0][0]       │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Concatenate1        │ (None, 120, 240,  │          0 │ Normalize_DBZ[0]… │
│ (Concatenate)       │ 12)               │            │ Normalize_VEL[0]… │
│                     │                   │            │ Normalize_KDP[0]… │
│                     │                   │            │ Normalize_RHOHV[… │
│                     │                   │            │ Normalize_ZDR[0]… │
│                     │                   │            │ Normalize_WIDTH[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ isnan_1 (Isnan)     │ (None, 120, 240,  │          0 │ Concatenate1[0][… │
│                     │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ where_1 (Where)     │ (None, 120, 240,  │          0 │ isnan_1[0][0],    │
│                     │ 12)               │            │ Concatenate1[0][… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 8,084,225 (30.84 MB)

 Trainable params: 8,084,225 (30.84 MB)

 Non-trainable params: 0 (0.00 B)

Again, as mentioned above, of note here is the learning rate chosen, which is 10^-4.

The original version of this notebook attempted to use 10^-3, but that caused the model to fail to converge after just one epoch of training (later epochs failed to make any progress). In comparison, using 10^-4 (while slower initially) seems to work well in training the data over the course of at least four epochs.

As noted in the previous notebook, we might consider decaying the learning rate even further (due to our small batch size) later in this training for later epochs, because the small batch size can cause noisiness in gradient descent updates (so dampening those updates a bit can help the model converge). We'll monitor and see how that goes.

In [9]:
opt = keras.optimizers.Adam(learning_rate=1e-4)
loss = keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(loss=loss, optimizer=opt)

We create a checkpoint saving function just to deal with iPython's many kernel crashes.
After each pass through the data and all batches, we'll save the weights, reload the model, and keep going.

We'll save the weights in `checkpoints/epoch_{number_of_epoch}.weights.h5`

In [19]:
def checkpoint_creator(checkpoint_path):
    # saving the model's weights in case the iPython kernel crashes (which it likes to do)
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                       save_weights_only=True,
                                       verbose=1)
    return cp_callback

def checkpoint_loader(checkpoint_path, model):
    model.load_weights(checkpoint_path)

First pass through the data. Again, with each pass, we set it up with:
- A batch_size of 64 (to fit the data into GPU memory)
- Shuffle the data with a random_state (we pick a new seed each time, but record that seed here so we don't reuse it in a later epoch)
- Set up a new checkpoint for each epoch (in case any one epoch of training crashes/fails).

In [12]:
preprocessed = make_tf_with_flips_loader(data_root = TORNET_DATA_INPUT_FOLDER, 
                                         data_type = "train", # or 'test'
                                         years = list(range(2013, 2023)),
                                         batch_size = 64, 
                                         weights = None,
                                         include_az = False,
                                         random_state = 5678,
                                         select_keys = ALL_VARIABLES + ["coordinates", "range_folded_mask"],
                                         tilt_last = True,
                                         from_tfds = False,
                                         tfds_data_version ="1.1.0")

In [13]:
model.fit(preprocessed, epochs=1, callbacks=[checkpoint_creator("checkpoints/epoch_flipping_1_1e-4.weights.h5")])

I0000 00:00:1731973690.907013   17376 service.cc:148] XLA service 0x7f62f8019090 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731973690.907543   17376 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-11-18 15:48:11.049961: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1731973691.398709   17376 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1731973695.090274   17376 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
I0000 00:00:1731973715.098554   17376 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


   2867/Unknown 9308s 3s/step - loss: 1.3145
Epoch 1: saving model to checkpoints/epoch_flipping_1_1e-4.weights.h5


2024-11-18 18:23:15.488076: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-11-18 18:23:15.490720: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_12]]
2024-11-18 18:23:15.491729: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 18317437428072606055
2024-11-18 18:23:15.491761: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 12506195242215264983
2024-11-18 18:23:15.491768: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 944059850056747535
2024-11-18 18:23:15.491772: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 1264007328355077277
2024-11-1

2867/2867 ━━━━━━━━━━━━━━━━━━━━ 9310s 3s/step - loss: 1.3143


Now we loop through a bunch of epochs to train the CNN further, making sure to change the random_state each time to mini-batch the data differently each time.

In [ ]:
for epoch_num in range(2, 5):
    resampled = make_tf_with_flips_loader(data_root = TORNET_DATA_INPUT_FOLDER, 
                                          data_type = "train", # or 'test'
                                          years = list(range(2013, 2023)),
                                          batch_size = 64, 
                                          weights = None,
                                          include_az = False,
                                          random_state = 1234 + epoch_num,
                                          select_keys = ALL_VARIABLES + ["coordinates", "range_folded_mask"],
                                          tilt_last = True,
                                          from_tfds = False,
                                          tfds_data_version ="1.1.0")
    checkpoint_loader("checkpoints/epoch_flipping_{}_1e-4.weights.h5".format(str(epoch_num - 1)), model)
    model.fit(resampled, epochs=1, callbacks=[checkpoint_creator("checkpoints/epoch_flipping_{}_1e-4.weights.h5".format(epoch_num))])

   2867/Unknown 9357s 3s/step - loss: 0.3107
Epoch 1: saving model to checkpoints/epoch_flipping_2_1e-4.weights.h5


2024-11-18 23:22:16.192384: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_12]]
2024-11-18 23:22:16.197645: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 18317437428072606055
2024-11-18 23:22:16.197671: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 12506195242215264983
2024-11-18 23:22:16.197679: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 944059850056747535
2024-11-18 23:22:16.197684: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 1264007328355077277
2024-11-18 23:22:16.197716: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 4262657239284205003
2024-11-18 23:22:16.197746: I tensorflow/cor

2867/2867 ━━━━━━━━━━━━━━━━━━━━ 9359s 3s/step - loss: 0.3107


2024-11-19 01:40:10.174991: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 117964864 bytes after encountering the first element of size 117964864 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


      3/Unknown 34s 14s/step - loss: 0.2262

In [11]:
for epoch_num in range(3, 5):
    resampled = make_tf_with_flips_loader(data_root = TORNET_DATA_INPUT_FOLDER, 
                                          data_type = "train", # or 'test'
                                          years = list(range(2013, 2023)),
                                          batch_size = 64, 
                                          weights = None,
                                          include_az = False,
                                          random_state = 1234 + epoch_num,
                                          select_keys = ALL_VARIABLES + ["coordinates", "range_folded_mask"],
                                          tilt_last = True,
                                          from_tfds = False,
                                          tfds_data_version ="1.1.0")
    checkpoint_loader("checkpoints/epoch_flipping_{}_1e-4.weights.h5".format(str(epoch_num - 1)), model)
    model.fit(resampled, epochs=1, callbacks=[checkpoint_creator("checkpoints/epoch_flipping_{}_1e-4.weights.h5".format(epoch_num))])

/usr/local/lib/python3.9/dist-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 54 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
I0000 00:00:1732045742.183686   21628 service.cc:148] XLA service 0x7f6628018f50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732045742.184072   21628 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-11-19 11:49:02.310836: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1732045742.629618   21628 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1732045765.958023   21628 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


   2867/Unknown 9839s 3s/step - loss: 0.2538
Epoch 1: saving model to checkpoints/epoch_flipping_3_1e-4.weights.h5


2024-11-19 14:32:58.471917: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-11-19 14:32:58.472487: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_18]]
2024-11-19 14:32:58.472792: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 18125960373766345697
2024-11-19 14:32:58.472815: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 9072318686438974239
2024-11-19 14:32:58.472821: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 1040842881928324157
2024-11-19 14:32:58.472825: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 1237740458513806647
2024-11-1

2867/2867 ━━━━━━━━━━━━━━━━━━━━ 9842s 3s/step - loss: 0.2538
   2867/Unknown 11148s 4s/step - loss: 0.2376
Epoch 1: saving model to checkpoints/epoch_flipping_4_1e-4.weights.h5


2024-11-19 19:56:04.202585: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_18]]
/usr/lib/python3.9/contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-11-19 19:56:04.206253: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 18125960373766345697
2024-11-19 19:56:04.206311: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 9072318686438974239
2024-11-19 19:56:04.206324: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 1040842881928324157
2024-11-19 19:56:04.206328: 

2867/2867 ━━━━━━━━━━━━━━━━━━━━ 11151s 4s/step - loss: 0.2376


In [ ]:
for epoch_num in range(5, 7):
    resampled = make_tf_with_flips_loader(data_root = TORNET_DATA_INPUT_FOLDER, 
                                          data_type = "train", # or 'test'
                                          years = list(range(2013, 2023)),
                                          batch_size = 64, 
                                          weights = None,
                                          include_az = False,
                                          random_state = 1234 + epoch_num,
                                          select_keys = ALL_VARIABLES + ["coordinates", "range_folded_mask"],
                                          tilt_last = True,
                                          from_tfds = False,
                                          tfds_data_version ="1.1.0")
    checkpoint_loader("checkpoints/epoch_flipping_{}_1e-4.weights.h5".format(str(epoch_num - 1)), model)
    model.fit(resampled, epochs=1, callbacks=[checkpoint_creator("checkpoints/epoch_flipping_{}_1e-4.weights.h5".format(epoch_num))])

In [ ]:
for epoch_num in range(7, 8):
    resampled = make_tf_with_flips_loader(data_root = TORNET_DATA_INPUT_FOLDER, 
                                          data_type = "train", # or 'test'
                                          years = list(range(2013, 2023)),
                                          batch_size = 64, 
                                          weights = None,
                                          include_az = False,
                                          random_state = 1234 + epoch_num,
                                          select_keys = ALL_VARIABLES + ["coordinates", "range_folded_mask"],
                                          tilt_last = True,
                                          from_tfds = False,
                                          tfds_data_version ="1.1.0")
    checkpoint_loader("checkpoints/epoch_flipping_{}_1e-4.weights.h5".format(str(epoch_num - 1)), model)
    model.fit(resampled, epochs=1, callbacks=[checkpoint_creator("checkpoints/epoch_flipping_{}_1e-4.weights.h5".format(epoch_num))])

/usr/local/lib/python3.9/dist-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 54 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


   2867/Unknown 9105s 3s/step - loss: 0.2140
Epoch 1: saving model to checkpoints/epoch_flipping_7_1e-4.weights.h5


2024-11-20 20:50:42.094627: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_8]]
2024-11-20 20:50:42.097826: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 11792009927715927949
2024-11-20 20:50:42.097844: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 10503351137423630075
2024-11-20 20:50:42.097849: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 8873016923517038427
2024-11-20 20:50:42.097855: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 1358983975771822302
2024-11-20 20:50:42.097859: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 5465932250389205152
2024-11-20 20:50:42.097862: I tensorflow/cor

2867/2867 ━━━━━━━━━━━━━━━━━━━━ 9107s 3s/step - loss: 0.2140


2024-11-20 23:03:57.895288: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 117964864 bytes after encountering the first element of size 117964864 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


      4/Unknown 18s 4s/step - loss: 0.1937

In [10]:
for epoch_num in range(8, 9):
    resampled = make_tf_with_flips_loader(data_root = TORNET_DATA_INPUT_FOLDER, 
                                          data_type = "train", # or 'test'
                                          years = list(range(2013, 2023)),
                                          batch_size = 64, 
                                          weights = None,
                                          include_az = False,
                                          random_state = 1234 + epoch_num,
                                          select_keys = ALL_VARIABLES + ["coordinates", "range_folded_mask"],
                                          tilt_last = True,
                                          from_tfds = False,
                                          tfds_data_version ="1.1.0")
    checkpoint_loader("checkpoints/epoch_flipping_{}_1e-4.weights.h5".format(str(epoch_num - 1)), model)
    model.fit(resampled, epochs=1, callbacks=[checkpoint_creator("checkpoints/epoch_flipping_{}_1e-4.weights.h5".format(epoch_num))])

/usr/local/lib/python3.9/dist-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 54 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
I0000 00:00:1732181742.860156   15124 service.cc:148] XLA service 0x7f3e9c018eb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732181742.860471   15124 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-11-21 01:35:42.973705: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1732181743.275769   15124 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1732181745.413249   15124 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, pl

   2867/Unknown 9033s 3s/step - loss: 0.2079
Epoch 1: saving model to checkpoints/epoch_flipping_8_1e-4.weights.h5


2024-11-21 04:06:12.691764: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-11-21 04:06:12.691856: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_10]]
2024-11-21 04:06:12.691868: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 12827997819925368895
2024-11-21 04:06:12.691871: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 10259843889643680281
2024-11-21 04:06:12.691876: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 2283971990208613307
2024-11-21 04:06:12.691879: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 65729284537371621
2024-11-21

2867/2867 ━━━━━━━━━━━━━━━━━━━━ 9035s 3s/step - loss: 0.2079


We checked the validation AUC's as we went along in order to make sure that AUC was increasing with each epoch of training,
and to make adjustments to hyperparameters if we were stalling out at any point.

After epochs 7 and 8, we noticed that there wasn't much of an improvement from epoch 7 to epoch 8 in the validation AUC.
Hence, we theorized that there might be a learning rate issue (maybe the learning rate is too large to make improvements),
so we reduced the learning rate from 10^-4 to 5 * 10^-5, cutting the learning rate in half and hoping that would help
as the loss begins to converge.

In [20]:
opt = keras.optimizers.Adam(learning_rate=5e-5)
loss = keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(loss=loss, optimizer=opt)

for epoch_num in range(9, 10):
    resampled = make_tf_with_flips_loader(data_root = TORNET_DATA_INPUT_FOLDER, 
                                          data_type = "train", # or 'test'
                                          years = list(range(2013, 2023)),
                                          batch_size = 64, 
                                          weights = None,
                                          include_az = False,
                                          random_state = 1234 + epoch_num,
                                          select_keys = ALL_VARIABLES + ["coordinates", "range_folded_mask"],
                                          tilt_last = True,
                                          from_tfds = False,
                                          tfds_data_version ="1.1.0")
    checkpoint_loader("checkpoints/epoch_flipping_{}_1e-4.weights.h5".format(str(epoch_num - 1)), model)
    model.fit(resampled, epochs=1, callbacks=[checkpoint_creator("checkpoints/epoch_flipping_{}_1e-4.weights.h5".format(epoch_num))])

/usr/local/lib/python3.9/dist-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 54 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


   2867/Unknown 14114s 5s/step - loss: 0.1976

2024-11-21 15:08:15.938919: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_10]]
2024-11-21 15:08:15.942789: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 12827997819925368895
2024-11-21 15:08:15.942815: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 10259843889643680281
2024-11-21 15:08:15.942823: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 2283971990208613307
2024-11-21 15:08:15.942828: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 65729284537371621
2024-11-21 15:08:15.942839: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 4687782750711658532
2024-11-21 15:08:15.942843: I tensorflow/core


Epoch 1: saving model to checkpoints/epoch_flipping_9_1e-4.weights.h5
2867/2867 ━━━━━━━━━━━━━━━━━━━━ 14135s 5s/step - loss: 0.1976


## Testing on test data

In our test data, we DON'T want to do any flipping or data augmentation. So, before we run the test data 
evaluation, we have to bring back TorNet's original functions for loading the test data into a 
Tensorflow dataset.

After we do that, we can run through the testing data for each epoch to see how the model performed
after each epoch of training in terms of AUC.

In [10]:
def create_tf_dataset(files:str,
                      variables: List[str]=ALL_VARIABLES,
                      n_frames:int=1,
                      tilt_last: bool=True) -> tf.data.Dataset:
    """
    This is Tornet's main function for loading the data from the folder where it's all stored.
    
    As they stated, this creates a TF dataset object via the function read_file (which reads the NetCDF files
    into the data one at a time).
    """
    assert len(files)>0
    # grab one file to gets keys, shapes, etc
    data = read_file(files[0],variables=variables,n_frames=n_frames, tilt_last=tilt_last)
    
    output_signature = { k:tf.TensorSpec(shape=data[k].shape,dtype=data[k].dtype,name=k) for k in data }
    def gen():
        for f in files:
            yield read_file(f,variables=variables,n_frames=n_frames, tilt_last=tilt_last)
    ds = tf.data.Dataset.from_generator(gen,
                                        output_signature=output_signature)
    return ds

def make_tf_loader(data_root: str, 
            data_type:str='train', # or 'test'
            years: list=list(range(2013,2023)),
            batch_size: int=128, 
            weights: Dict=None,
            include_az: bool=False,
            random_state:int=1234,
            select_keys: list=None,
            tilt_last: bool=True,
            from_tfds: bool=False,
            tfds_data_version: str='1.1.0',
            num_epochs: int=3):
    """
    This TorNet library function is used to load the data into Tensorflow.
    We're going to use the `create_tf_dataset` function from above, 
    then we'll use `preproc` to preprocess it.

    One important note - we tried a bunch of different functions for shuffling 
    and batching, repeating the dataset, etc. to try to be able to run
    many epochs with one function call. It wasn't working.
    Even the `drop_remainder=True` that we've added here to ds.batch
    seems to not really have an effect, as the model training
    still throws an error at the end of training about running out of data.
    """
    
    if from_tfds: # fast loader
        import tensorflow_datasets as tfds
        import tornet.data.tfds.tornet.tornet_dataset_builder # registers 'tornet'
        ds = tfds.load('tornet:%s' % tfds_data_version ,split='+'.join(['%s-%d' % (data_type,y) for y in years]))
        # Assumes data was saved with tilt_last=True and converts it to tilt_last=False
        if not tilt_last:
            ds = ds.map(lambda d: pp.permute_dims(d,(0,3,1,2), backend=tf))
    else: # Load directly from netcdf files
        file_list = query_catalog(data_root, data_type, years, random_state)
        ds = create_tf_dataset(file_list,variables=ALL_VARIABLES,n_frames=1, tilt_last=tilt_last) 

    ds = preproc(ds,weights,include_az,select_keys,tilt_last)
    ds = ds.prefetch(tf.data.AUTOTUNE)

    # this has been adjusted to include drop_remainder=True
    ds = ds.batch(batch_size, drop_remainder=True)
    return ds

For each epoch that we trained through, let's test the model and see how it did after each pass of the data.
AUC is our evaluation metric here.

In [11]:
test_data = make_tf_loader(data_root = TORNET_DATA_INPUT_FOLDER, 
                              data_type = "test",
                              years = list(range(2013, 2023)),
                              batch_size = 64, 
                              weights = None,
                              include_az = False,
                              random_state = 5678,
                              select_keys = ALL_VARIABLES + ["coordinates", "range_folded_mask"],
                              tilt_last = True,
                              from_tfds = False,
                              tfds_data_version ="1.1.0")

for epoch_num in range(3, 5):
    metrics = [keras.metrics.AUC(from_logits=True,name='AUC')]
    checkpoint_loader("checkpoints/epoch_flipping_{}_1e-4.weights.h5".format(str(epoch_num)), model)
    model.compile(loss=loss, metrics=metrics)
    model.evaluate(test_data)

/usr/local/lib/python3.9/dist-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 54 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
I0000 00:00:1732079977.799323     856 service.cc:148] XLA service 0x7ff4a800db50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732079977.799664     856 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-11-19 21:19:37.834266: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1732079977.932640     856 cuda_dnn.cc:529] Loaded cuDNN version 90300


      2/Unknown 13s 67ms/step - AUC: 0.4362 - loss: 0.1598   

I0000 00:00:1732079986.378947     856 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


491/491 ━━━━━━━━━━━━━━━━━━━━ 1590s 3s/step - AUC: 0.8184 - loss: 0.2154


2024-11-19 21:46:03.694408: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-11-19 21:46:03.694462: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_14]]
2024-11-19 21:46:03.694472: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 12744976677381798467
2024-11-19 21:46:03.694475: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 9096861446359773735
2024-11-19 21:46:03.694479: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 5502915226632464905
2024-11-19 21:46:03.694482: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 8169583992736786955
2024-11-1

491/491 ━━━━━━━━━━━━━━━━━━━━ 1668s 3s/step - AUC: 0.8388 - loss: 0.2028


2024-11-19 22:13:52.101356: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_14]]
/usr/lib/python3.9/contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-11-19 22:13:52.101397: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 9799722824554614607
2024-11-19 22:13:52.101403: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 9096861446359773735
2024-11-19 22:13:52.101407: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 8169583992736786955
2024-11-19 22:13:52.101410: I

In [11]:
test_data = make_tf_loader(data_root = TORNET_DATA_INPUT_FOLDER, 
                              data_type = "test",
                              years = list(range(2013, 2023)),
                              batch_size = 64, 
                              weights = None,
                              include_az = False,
                              random_state = 5678,
                              select_keys = ALL_VARIABLES + ["coordinates", "range_folded_mask"],
                              tilt_last = True,
                              from_tfds = False,
                              tfds_data_version ="1.1.0")

for epoch_num in range(6, 8):
    metrics = [keras.metrics.AUC(from_logits=True,name='AUC')]
    checkpoint_loader("checkpoints/epoch_flipping_{}_1e-4.weights.h5".format(str(epoch_num)), model)
    model.compile(loss=loss, metrics=metrics)
    model.evaluate(test_data)

/usr/local/lib/python3.9/dist-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 54 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
I0000 00:00:1732135689.176521    4987 service.cc:148] XLA service 0x7fae94003af0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732135689.176917    4987 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-11-20 12:48:09.210472: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1732135689.302695    4987 cuda_dnn.cc:529] Loaded cuDNN version 90300


      2/Unknown 13s 74ms/step - AUC: 0.4285 - loss: 0.1316   

I0000 00:00:1732135697.837069    4987 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


491/491 ━━━━━━━━━━━━━━━━━━━━ 1533s 3s/step - AUC: 0.8609 - loss: 0.1914


2024-11-20 13:13:38.874242: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-11-20 13:13:38.874306: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_8]]
2024-11-20 13:13:38.874324: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 8873016923517038427
2024-11-20 13:13:38.874329: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 10503351137423630075
2024-11-20 13:13:38.874334: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 11792009927715927949
2024-11-20 13:13:38.874339: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 1358983975771822302
2024-11-2

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'checkpoints/epoch_flipping_7_1e-4.weights.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [12]:
test_data = make_tf_loader(data_root = TORNET_DATA_INPUT_FOLDER, 
                              data_type = "test",
                              years = list(range(2013, 2023)),
                              batch_size = 64, 
                              weights = None,
                              include_az = False,
                              random_state = 5678,
                              select_keys = ALL_VARIABLES + ["coordinates", "range_folded_mask"],
                              tilt_last = True,
                              from_tfds = False,
                              tfds_data_version ="1.1.0")

for epoch_num in range(7, 9):
    metrics = [keras.metrics.AUC(from_logits=True,name='AUC')]
    checkpoint_loader("checkpoints/epoch_flipping_{}_1e-4.weights.h5".format(str(epoch_num)), model)
    model.compile(loss=loss, metrics=metrics)
    model.evaluate(test_data)

491/491 ━━━━━━━━━━━━━━━━━━━━ 1607s 3s/step - AUC: 0.8659 - loss: 0.1903


2024-11-21 08:35:31.749291: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_10]]
2024-11-21 08:35:31.750965: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 12827997819925368895
2024-11-21 08:35:31.750981: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 10259843889643680281
2024-11-21 08:35:31.750989: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 2283971990208613307
2024-11-21 08:35:31.750994: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 65729284537371621
2024-11-21 08:35:31.751002: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 4687782750711658532
2024-11-21 08:35:31.751006: I tensorflow/core

491/491 ━━━━━━━━━━━━━━━━━━━━ 1610s 3s/step - AUC: 0.8663 - loss: 0.1890


2024-11-21 09:02:22.190477: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 12827997819925368895
/usr/lib/python3.9/contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-11-21 09:02:22.190519: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 10259843889643680281
2024-11-21 09:02:22.190526: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 2283971990208613307
2024-11-21 09:02:22.190532: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 65729284537371621
2024-11-21 09:02:22.190539: I tensorflow/core/framework/local_rendezvous.cc:424] Local ren

In [11]:
test_data = make_tf_loader(data_root = TORNET_DATA_INPUT_FOLDER, 
                              data_type = "test",
                              years = list(range(2013, 2023)),
                              batch_size = 64, 
                              weights = None,
                              include_az = False,
                              random_state = 5678,
                              select_keys = ALL_VARIABLES + ["coordinates", "range_folded_mask"],
                              tilt_last = True,
                              from_tfds = False,
                              tfds_data_version ="1.1.0")

for epoch_num in range(9, 10):
    metrics = [keras.metrics.AUC(from_logits=True,name='AUC')]
    checkpoint_loader("checkpoints/epoch_flipping_{}_1e-4.weights.h5".format(str(epoch_num)), model)
    model.compile(loss=loss, metrics=metrics)
    model.evaluate(test_data)

/usr/local/lib/python3.9/dist-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 54 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
I0000 00:00:1732231767.728496   23418 service.cc:148] XLA service 0x7fce5000d3c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732231767.729281   23418 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-11-21 15:29:27.773098: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1732231767.896476   23418 cuda_dnn.cc:529] Loaded cuDNN version 90300


      2/Unknown 11s 65ms/step - AUC: 0.4374 - loss: 0.1248   

I0000 00:00:1732231774.914547   23418 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


491/491 ━━━━━━━━━━━━━━━━━━━━ 1546s 3s/step - AUC: 0.8804 - loss: 0.1836


2024-11-21 15:55:09.686421: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-11-21 15:55:09.686505: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_14]]
2024-11-21 15:55:09.686522: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 13487443787624175879
2024-11-21 15:55:09.686527: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 6682284121449466355
2024-11-21 15:55:09.686532: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 6907914624901376875
2024-11-21 15:55:09.686535: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 17021916078633971215
2024-11-

## Significant results!

This above result (of a validation AUC of .880) is already massive. By using our augmented dataset to train the data (and give it more tornadic examples), we've increased the performance of the TorNet model without any error analysis on the validation set (basically, "without peeking").

Note that none of the test/validation examples are flipped. Only our training examples included flipped tornadoes. This means that even though the flipped tornadoes aren't necessarily "real" tornadoes, flipped tornadoes are good enough examples to help the CNN architecture learn what a tornado looks like!

Our intuition that the class imbalance was causing issues seems to have been validated - even without five-fold cross validation, the model we've trained with the augmented dataset has a better AUC than the original CNN model that was used. With a validation AUC of .8804, we've beaten the benchmark that TorNet set with their CNN (which, when published, had achieved a validation AUC of .874 at its best). 

The idea to also decrease the learning rate for the last couple of iterations also seems to have helped here. Between epochs 7 and 8 of training (with a learning rate of 10^-4), we saw hardly any improvement in the AUC. So, we used a learning rate of 5 * 10^-5 for epoch 9 of training, and got a massive AUC improvement (probably due to dampening the noisy updates now that we're "closer" to converging).

The best part is that there's still a chance for further improvement here with more training, which we'll attempt below. We might even consider further reducing the learning rate if we don't see any improvements on these next epochs of training, just to see if we can squeeze out any more improvements.

## Running further epochs on the data

Let's run more epochs with this adjusted learning rate and test the model's performance on the test validation set after each epoch.

Our plan is to continue using decaying learning rate and early stopping in order to get the best model possible without overfitting or overtuning - once we see that validation AUC is not improving, we decrease learning rate, and if we see that even a decreased learning rate doesn't improve the performance, we stop training more epochs.

In [21]:
opt = keras.optimizers.Adam(learning_rate=1e-5)
loss = keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(loss=loss, optimizer=opt)

for epoch_num in range(10, 11):
    resampled = make_tf_with_flips_loader(data_root = TORNET_DATA_INPUT_FOLDER, 
                                          data_type = "train", # or 'test'
                                          years = list(range(2013, 2023)),
                                          batch_size = 64, 
                                          weights = None,
                                          include_az = False,
                                          random_state = 1234 + epoch_num,
                                          select_keys = ALL_VARIABLES + ["coordinates", "range_folded_mask"],
                                          tilt_last = True,
                                          from_tfds = False,
                                          tfds_data_version ="1.1.0")
    checkpoint_loader("checkpoints/epoch_flipping_{}_1e-4.weights.h5".format(str(epoch_num - 1)), model)
    model.fit(resampled, epochs=1, callbacks=[checkpoint_creator("checkpoints/epoch_flipping_{}_1e-5.weights.h5".format(epoch_num))])

/usr/local/lib/python3.9/dist-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 54 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


   2867/Unknown 9457s 3s/step - loss: 0.1893
Epoch 1: saving model to checkpoints/epoch_flipping_10_1e-5.weights.h5


2024-11-22 13:23:46.313413: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 9826376891565193259
2024-11-22 13:23:46.315518: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 13217425807873929449
2024-11-22 13:23:46.315531: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 9430844678276428555
2024-11-22 13:23:46.315536: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 10349365694754166267
2024-11-22 13:23:46.315692: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 13923780728266016189
2024-11-22 13:23:46.315715: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 6329571622297899138
2024-11-22 13:23:46.315721: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv 

2867/2867 ━━━━━━━━━━━━━━━━━━━━ 9459s 3s/step - loss: 0.1893


In [12]:
test_data = make_tf_loader(data_root = TORNET_DATA_INPUT_FOLDER, 
                              data_type = "test",
                              years = list(range(2013, 2023)),
                              batch_size = 64, 
                              weights = None,
                              include_az = False,
                              random_state = 5678,
                              select_keys = ALL_VARIABLES + ["coordinates", "range_folded_mask"],
                              tilt_last = True,
                              from_tfds = False,
                              tfds_data_version ="1.1.0")

for epoch_num in range(10, 11):
    metrics = [keras.metrics.AUC(from_logits=True,name='AUC')]
    checkpoint_loader("checkpoints/epoch_flipping_{}_1e-4.weights.h5".format(str(epoch_num)), model)
    model.compile(loss=loss, metrics=metrics)
    model.evaluate(test_data)

491/491 ━━━━━━━━━━━━━━━━━━━━ 1540s 3s/step - AUC: 0.8781 - loss: 0.1824


/usr/lib/python3.9/contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-11-22 08:22:17.536217: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_4]]
2024-11-22 08:22:17.536269: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 9826376891565193259
2024-11-22 08:22:17.536275: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 13217425807873929449
2024-11-22 08:22:17.536280: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 9430844678276428555
2024-11-22 08:22:17.536284: I

In [23]:
test_data = make_tf_loader(data_root = TORNET_DATA_INPUT_FOLDER, 
                              data_type = "test",
                              years = list(range(2013, 2023)),
                              batch_size = 64, 
                              weights = None,
                              include_az = False,
                              random_state = 5678,
                              select_keys = ALL_VARIABLES + ["coordinates", "range_folded_mask"],
                              tilt_last = True,
                              from_tfds = False,
                              tfds_data_version ="1.1.0")

for epoch_num in range(10, 11):
    metrics = [keras.metrics.AUC(from_logits=True,name='AUC')]
    checkpoint_loader("checkpoints/epoch_flipping_{}_1e-5.weights.h5".format(str(epoch_num)), model)
    model.compile(loss=loss, metrics=metrics)
    model.evaluate(test_data)

491/491 ━━━━━━━━━━━━━━━━━━━━ 1711s 3s/step - AUC: 0.8821 - loss: 0.1819


2024-11-22 14:00:35.525271: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_4]]
2024-11-22 14:00:35.526461: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 9826376891565193259
2024-11-22 14:00:35.526473: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 13217425807873929449
2024-11-22 14:00:35.526478: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 9430844678276428555
2024-11-22 14:00:35.526482: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 10349365694754166267
2024-11-22 14:00:35.526485: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 13923780728266016189
2024-11-22 14:00:35.526493: I tensorflow/co

Now we have achieved an AUC of .8821 (with different weights, and still better than the TorNet performance!). Is it possible to improve the model even further?

In [9]:
opt = keras.optimizers.Adam(learning_rate=1e-5)
loss = keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(loss=loss, optimizer=opt)

for epoch_num in range(11, 12):
    resampled = make_tf_with_flips_loader(data_root = TORNET_DATA_INPUT_FOLDER, 
                                          data_type = "train", # or 'test'
                                          years = list(range(2013, 2023)),
                                          batch_size = 64, 
                                          weights = None,
                                          include_az = False,
                                          random_state = 1234 + epoch_num,
                                          select_keys = ALL_VARIABLES + ["coordinates", "range_folded_mask"],
                                          tilt_last = True,
                                          from_tfds = False,
                                          tfds_data_version ="1.1.0")
    checkpoint_loader("checkpoints/epoch_flipping_{}_1e-5.weights.h5".format(str(epoch_num - 1)), model)
    model.fit(resampled, epochs=1, callbacks=[checkpoint_creator("checkpoints/epoch_flipping_{}_1e-5.weights.h5".format(epoch_num))])

/usr/local/lib/python3.9/dist-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 54 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
I0000 00:00:1732354833.996844     164 service.cc:148] XLA service 0x7f49f80190a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732354833.997136     164 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-11-23 01:40:34.112307: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1732354834.426541     164 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1732354836.374730     164 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, pl

   2867/Unknown 9069s 3s/step - loss: 0.1880
Epoch 1: saving model to checkpoints/epoch_flipping_11_1e-5.weights.h5


2024-11-23 04:11:39.798536: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-11-23 04:11:39.798572: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_6]]
2024-11-23 04:11:39.798582: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 130575521149143211
2024-11-23 04:11:39.798586: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 12751758492268379335
2024-11-23 04:11:39.798591: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 13236180624781059707
2024-11-23 04:11:39.798596: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 18308500856403318264
2024-11-2

2867/2867 ━━━━━━━━━━━━━━━━━━━━ 9071s 3s/step - loss: 0.1880


In [11]:
test_data = make_tf_loader(data_root = TORNET_DATA_INPUT_FOLDER, 
                              data_type = "test",
                              years = list(range(2013, 2023)),
                              batch_size = 64, 
                              weights = None,
                              include_az = False,
                              random_state = 5678,
                              select_keys = ALL_VARIABLES + ["coordinates", "range_folded_mask"],
                              tilt_last = True,
                              from_tfds = False,
                              tfds_data_version ="1.1.0")

for epoch_num in range(11, 12):
    metrics = [keras.metrics.AUC(from_logits=True,name='AUC')]
    checkpoint_loader("checkpoints/epoch_flipping_{}_1e-5.weights.h5".format(str(epoch_num)), model)
    model.compile(loss=loss, metrics=metrics)
    model.evaluate(test_data)

491/491 ━━━━━━━━━━━━━━━━━━━━ 1555s 3s/step - AUC: 0.8846 - loss: 0.1804


/usr/lib/python3.9/contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-11-23 10:07:47.012139: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_10]]
2024-11-23 10:07:47.012567: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 130575521149143211
2024-11-23 10:07:47.012575: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 12751758492268379335
2024-11-23 10:07:47.012580: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 13236180624781059707
2024-11-23 10:07:47.012587: 

.8846 AUC is even better. We've now beaten the original AUC of the baseline CNN by .010.

In [20]:
opt = keras.optimizers.Adam(learning_rate=1e-5)
loss = keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(loss=loss, optimizer=opt)

for epoch_num in range(12, 13):
    resampled = make_tf_with_flips_loader(data_root = TORNET_DATA_INPUT_FOLDER, 
                                          data_type = "train", # or 'test'
                                          years = list(range(2013, 2023)),
                                          batch_size = 64, 
                                          weights = None,
                                          include_az = False,
                                          random_state = 1234 + epoch_num,
                                          select_keys = ALL_VARIABLES + ["coordinates", "range_folded_mask"],
                                          tilt_last = True,
                                          from_tfds = False,
                                          tfds_data_version ="1.1.0")
    checkpoint_loader("checkpoints/epoch_flipping_{}_1e-5.weights.h5".format(str(epoch_num - 1)), model)
    model.fit(resampled, epochs=1, callbacks=[checkpoint_creator("checkpoints/epoch_flipping_{}_1e-5.weights.h5".format(epoch_num))])

/usr/local/lib/python3.9/dist-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 54 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


   2867/Unknown 9405s 3s/step - loss: 0.1860
Epoch 1: saving model to checkpoints/epoch_flipping_12_1e-5.weights.h5


2024-11-23 15:06:40.504714: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 130575521149143211
2024-11-23 15:06:40.507329: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 12751758492268379335
2024-11-23 15:06:40.507346: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 13236180624781059707
2024-11-23 15:06:40.507370: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 8644507059846065498
2024-11-23 15:06:40.507377: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 14579245408181732044
2024-11-23 15:06:40.507512: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 14547304850208728136
2024-11-23 15:06:40.507524: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv 

2867/2867 ━━━━━━━━━━━━━━━━━━━━ 9408s 3s/step - loss: 0.1860


In [21]:
test_data = make_tf_loader(data_root = TORNET_DATA_INPUT_FOLDER, 
                              data_type = "test",
                              years = list(range(2013, 2023)),
                              batch_size = 64, 
                              weights = None,
                              include_az = False,
                              random_state = 5678,
                              select_keys = ALL_VARIABLES + ["coordinates", "range_folded_mask"],
                              tilt_last = True,
                              from_tfds = False,
                              tfds_data_version ="1.1.0")

for epoch_num in range(12, 13):
    metrics = [keras.metrics.AUC(from_logits=True,name='AUC')]
    checkpoint_loader("checkpoints/epoch_flipping_{}_1e-5.weights.h5".format(str(epoch_num)), model)
    model.compile(loss=loss, metrics=metrics)
    model.evaluate(test_data)

491/491 ━━━━━━━━━━━━━━━━━━━━ 1626s 3s/step - AUC: 0.8854 - loss: 0.1806


2024-11-23 15:37:57.456046: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_10]]
2024-11-23 15:37:57.458470: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 130575521149143211
2024-11-23 15:37:57.458483: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 12751758492268379335
2024-11-23 15:37:57.458489: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 13236180624781059707
2024-11-23 15:37:57.458498: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 8644507059846065498
2024-11-23 15:37:57.458502: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 14579245408181732044
2024-11-23 15:37:57.458506: I tensorflow/co

## Conclusion

Ok, we're probably in a reasonable place to stop given that validation loss has stopped decreasing even with our decayed learning rate. 
With early stopping as our guideline (so that we don't end up overfitting), we should mark this as our best validation AUC.

We observed high validation AUC's for several epochs along the training (which we could, in some sense, consider to be separate models) with different sets of weights which all beat the original .8742 benchmark - epochs 9, 10, 11, and 12 all had validation AUC's that were .878 or better in the validation AUC. This is strong evidence that there are a number of possible trained models (using the augmented dataset) that are better than the original model trained on just the original dataset.

What does this mean for us? There are a few points to add here as important notes:
- First, flipping the tornadoes to create augmented data did NOT adversely impact the model that was trained. This was a concern that we had initially (is there weird "directionality" or "spin direction" that is lost when we flip the tornado?) but it proved to not be the case, and we achieved great success here.
- In fact, flipping the tornado images to create augmented data (as mentioned above) is probably a great strategy to 'create' more tornadoes and help correct for class imbalance caused by a relative lack of tornado data.
- In addition, what this shows above all is that CNN models trained on the dataset could do even better if we collect more tornado examples. Future research should not only use mirroring to augment the dataset, but should also focus on expanding the dataset to include more examples of labeled tornadoes. If anything, while the success of this solution shows that flipping some data is a good stopgap measure given a lack of data, it also proves that just generally having more tornado examples would absolutely help boost performance (because our synthetic tornadoes are essentially nothing more than just additional examples of tornadoes that help correct class imbalance).